In [1]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model, Sequential, Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

import numpy as np
from keras.utils.np_utils import to_categorical
import glob
import os

from sgdr import SGDRScheduler

Using TensorFlow backend.


In [2]:
categories = ['0_buildings', '1_forest', '2_glacier', '3_mountain', '4_sea', '5_street']

```python
def load_data(file_path):
    x_train = []
    y_train = []
    
    images = glob.glob(file_path + '/**/*')
    for photo in images:
        img = image.load_img(photo, target_size=(299, 299))
        tr_x = image.img_to_array(img)
        tr_x = preprocess_input(tr_x)
        label = (photo.split('\\')[1])
        label_place = categories.index(label)
        
        x_train.append(tr_x)
        y_train.append(label_place)
        
    return np.array(x_train), to_categorical(y_train)
```

```python
x_train, y_train = load_data('C:/Datasets/scene-classification/data_augment/train/')
```

In [7]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,
                                         rotation_range=10,
                                         width_shift_range=0.25,
                                         height_shift_range=0.25,
                                         shear_range=0.01,
                                         zoom_range=[0.75, 1.25],
                                         horizontal_flip=True,
                                         vertical_flip=False,
                                         data_format='channels_last',
                                         fill_mode='reflect',
                                         brightness_range=[0.75, 1.25]
                                         )

In [4]:
train_generator = train_datagen.flow_from_directory(
    directory='C:/Datasets/scene-classification/data/train/',
    target_size=(299, 299),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 15780 images belonging to 6 classes.


In [5]:
def load_validation_data(file_path):
    x_val = []
    y_val = []
    
    images = glob.glob(file_path + '/**/*')
    for photo in images:
        img = image.load_img(photo, target_size=(299, 299))
        tr_x = image.img_to_array(img)
        tr_x = preprocess_input(tr_x)
        label = (photo.split('\\')[1])
        label_place = categories.index(label)
        
        x_val.append(tr_x)
        y_val.append(label_place)
        
    return np.array(x_val), to_categorical(y_val)

In [6]:
x_val, y_val = load_validation_data('C:/Datasets/scene-classification/data/valid/')

In [7]:
x_val.shape

(1254, 299, 299, 3)

In [10]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = 32

In [12]:
input_tensor = Input(shape=(299, 299, 3))
base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(299, 299, 3), pooling='avg', classes=1000)
for layer in base_model.layers[:-13]:
    layer.trainable = False
    
t = base_model(input_tensor)
output_tensor = Dense(len(categories), activation='softmax')(t)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 9222      
Total params: 54,345,958
Trainable params: 4,711,398
Non-trainable params: 49,634,560
_________________________________________________________________


In [9]:
model = load_model('C:/Datasets/scene-classification/models/irnv2_adam_sgdr-e16-v0.92-t0.92.hdf5')

In [13]:
file_path = 'C:/Datasets/scene-classification/models/'
model_name = 'irnv2_adam_sgdr_v0.2-e{epoch:02d}-v{val_categorical_accuracy:.2f}-t{categorical_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(file_path + model_name, monitor='val_categorical_accuracy', save_best_only=True, verbose=1, mode='max')
schedule = SGDRScheduler(min_lr=1e-5, max_lr=1e-2,
                         steps_per_epoch=STEP_SIZE_TRAIN,
                         lr_decay=0.9,
                         cycle_length=3,
                         mult_factor=1.5)
tboard = TensorBoard(log_dir='C:/Datasets/scene-classification/logs/')

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [14]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=(x_val, y_val),
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[schedule, checkpoint, tboard],
                    epochs=50,
                    verbose=1
                   )

Epoch 1/50
493/493 [==============================] - 369s 749ms/step - loss: 0.2565 - categorical_accuracy: 0.9071 - val_loss: 0.2657 - val_categorical_accuracy: 0.9203

Epoch 00001: val_categorical_accuracy improved from -inf to 0.92026, saving model to C:/Datasets/scene-classification/models/irnv2_adam_sgdr_v0.1-e01-v0.92-t0.91.hdf5
Epoch 2/50
493/493 [==============================] - 365s 741ms/step - loss: 0.2280 - categorical_accuracy: 0.9168 - val_loss: 0.2255 - val_categorical_accuracy: 0.9234

Epoch 00002: val_categorical_accuracy improved from 0.92026 to 0.92344, saving model to C:/Datasets/scene-classification/models/irnv2_adam_sgdr_v0.1-e02-v0.92-t0.92.hdf5
Epoch 3/50
493/493 [==============================] - 364s 739ms/step - loss: 0.1892 - categorical_accuracy: 0.9307 - val_loss: 0.2285 - val_categorical_accuracy: 0.9258

Epoch 00003: val_categorical_accuracy improved from 0.92344 to 0.92584, saving model to C:/Datasets/scene-classification/models/irnv2_adam_sgdr_v0.1-e